In [ ]:
# import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import numpy as np

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

# Data

In [ ]:
# create data

nPerClust = 300
blur = 1

A = [ 1, 1 ]
B = [ 5, 1 ]
C = [ 4, 3 ]

# generate data
a = [ A[0]+np.random.randn(nPerClust)*blur , A[1]+np.random.randn(nPerClust)*blur ]
b = [ B[0]+np.random.randn(nPerClust)*blur , B[1]+np.random.randn(nPerClust)*blur ]
c = [ C[0]+np.random.randn(nPerClust)*blur , C[1]+np.random.randn(nPerClust)*blur ]

# true labels
labels_np = np.hstack((  np.zeros((nPerClust)),
                         np.ones( (nPerClust)),
                       1+np.ones( (nPerClust))  ))

# concatanate into a matrix
data_np = np.hstack((a,b,c)).T

# convert to a pytorch tensor
data = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).long() # note: "long" format for CCE

# show the data
fig = plt.figure(figsize=(5,5))
plt.plot(data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs',alpha=.5)
plt.plot(data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ko',alpha=.5)
plt.plot(data[np.where(labels==2)[0],0],data[np.where(labels==2)[0],1],'r^',alpha=.5)
plt.title('The qwerties!')
plt.xlabel('qwerty dimension 1')
plt.ylabel('qwerty dimension 2')
plt.show()

In [ ]:
# use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(data, labels, test_size=.1)

# then convert them into PyTorch Datasets (note: already converted to tensors)
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# finally, translate into dataloader objects
batchsize    = 16
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

# Model

In [ ]:
# Create a class for the neural network model and associated functions.

# Define a class for the neural network model.
def createTheQwertyNet(momentum):
    # Define a custom class for the neural network model.
    class qwertyNet(nn.Module):
        def __init__(self):
            super().__init__()

            # Input layer: It has 2 input features and connects to 8 neurons.
            self.input = nn.Linear(2, 8)

            # Hidden layer: One hidden layer with 8 neurons.
            self.fc1 = nn.Linear(8, 8)

            # Output layer: Connects to 3 neurons for classification (3 classes).
            self.output = nn.Linear(8, 3)

        # Forward pass: Defines how data flows through the network.
        def forward(self, x):
            # Apply Rectified Linear Unit (ReLU) activation to the input layer.
            x = F.relu(self.input(x))
            
            # Apply ReLU activation to the hidden layer.
            x = F.relu(self.fc1(x))
            
            # Pass the result through the output layer.
            return self.output(x)

    # Create an instance of the neural network model.
    net = qwertyNet()

    # Define the loss function (Cross Entropy Loss) for classification tasks.
    lossfun = nn.CrossEntropyLoss()

    # Define the optimizer (Stochastic Gradient Descent) with specified learning rate and momentum.
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=momentum)

    # Return the created neural network, loss function, and optimizer.
    return net, lossfun, optimizer


In [ ]:
# Confirm that the optimizer has momentum

# Create an instance of the optimizer with specified momentum.
optim = createTheQwertyNet(0.9)[2]

# Output the optimizer instance.
optim


# Training

In [ ]:
# a function that trains the model


# number of epochs
numepochs = 50


def function2trainTheModel(momentum):
  
  # create a new model
  net,lossfun,optimizer = createTheQwertyNet(momentum)

  # initialize losses
  losses   = torch.zeros(numepochs)
  trainAcc = []
  testAcc  = []

  # loop over epochs
  for epochi in range(numepochs):

    # switch on training mode
    net.train()

    # loop over training data batches
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100 
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    net.eval()
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      
    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) ) 
  # end epochs

  # function output
  return trainAcc,testAcc,losses,net


# Now for the experiment!

In [ ]:
# Test different momentum values during training

# Explanation:
# In this code snippet, we are conducting an experiment to evaluate the impact of different momentum values during the training of a neural network model.
# The `momenta` list contains various momentum values (0, 0.5, 0.9, 0.95, 0.999) that we want to test.

# Initialize a results matrix to store training results.
results = np.zeros((numepochs, len(momenta), 3))

# Explanation:
# The `results` matrix is a multidimensional array with dimensions (numepochs, len(momenta), 3). It will be used to store training results, including losses, training accuracy, and test accuracy. Each dimension of the matrix corresponds to different aspects of the results.

# Test all momentum values on the same data with different model instances.
for idx, mom in enumerate(momenta):
    # Explanation:
    # Within this loop, we iterate through the `momenta` list, and for each momentum value (`mom`), we call a function (presumably `function2trainTheModel`) to train the neural network model with that specific momentum value.

    # Call a function to train the model with the specified momentum.
    trainAcc, testAcc, losses, net = function2trainTheModel(mom)
    
    # Explanation:
    # The training results, including training accuracy, test accuracy, and losses, are obtained from the training process using the specified momentum value. These results are then stored in the `results` matrix.

    # Store the training losses, training accuracy, and test accuracy in the results matrix.
    results[:, idx, 0] = losses  # Column 0 stores losses
    results[:, idx, 1] = trainAcc  # Column 1 stores training accuracy
    results[:, idx, 2] = testAcc  # Column 2 stores test accuracy

# Explanation:
# After the loop completes, the `results` matrix contains the training results for each tested momentum value. This experimentation helps us understand how different momentum values affect the training process of the neural network model.


In [ ]:
fig,ax = plt.subplots(1,3,figsize=(16,5))

for i in range(3):
  ax[i].plot(results[:,:,i])
  ax[i].legend(momenta)
  ax[i].set_xlabel('Epochs')
  if i==0:
    ax[i].set_ylabel('Loss')
  else:
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].set_ylim([20,100])

ax[0].set_title('Losses')
ax[1].set_title('Train')
ax[2].set_title('Test')

plt.show()

# Additional explorations

In [ ]:
# 1) Now that you see the results across a broad range of beta (momentum) parameters, try re-running the experiment
#    using a narrower range. For example, you don't need to test b=0 or b=.999.
# 
# 2) The beta parameter multiplies the learning rate in the computation (see formula in slides). That means that these
#    results will interact with the learning rate. Repeat the experiment using a different learning rate.
# 
# 3) If you wanted to test the relationship between momentum and learning rate in a full parametric experiment, how would
#    you set it up? Would you store the loss/accuracy over all epochs? 
# 